In [1]:
import sys
import os

sys.path.append(os.getcwd()[:-9])
from src.ops.operator import Operator
from src.ops.elemental import ElementalOperator
from src.ops.quarks import Quark
from src.ops.indexed import IndexedObject
from src.wick.contract import contract

In [4]:
q0=Quark(True,'u','s_0','c_0','t_3','x_3')
g0=IndexedObject('\\gamma_5',['s_0','s_1'])
q1=Quark(False,'d','s_1','c_0','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g0],[q0,q1])])

#TODO add charge factors qu and qd

elementals=[]
for pi2 in [['u',+2/3],['d',-1/3]]:                              # uu + dd
    for pi1 in [['u',+2/3],['d',-1/3]]:                          # uu + dd 
        for dirpi2 in [-1,+1]:                               # eq 17 of notes
            for dirpi1 in [-1,+1]:
                q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                if(dirpi1==-1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                    g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                elif(dirpi1==+1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2+a\hat{\mu}')
                    g1=IndexedObject('(1+\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}^{\dagger}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2')
    
                if(dirpi2==-1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                    g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                    
                elif(dirpi2==+1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1+a\hat{\\nu}')
                    g3=IndexedObject('(1+\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}^{\dagger}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1')
        
                # for printing out the insertion cleanly, just comment out the pions, then use the following two blocks.  
                q6=Quark(False,'u','s_6','c_3','t_0','x_0')
                g5=IndexedObject('\\gamma_5',['s_6','s_7'])
                q7=Quark(True,'d','s_7','c_3','t_0','x_0')
                
                coef=pi1[1]*pi2[1]*dirpi1*dirpi2
                elementals.append(ElementalOperator(coef,[g1,g2,g3,g4,g5],[q2,q3,q4,q5,q6,q7]))
                #elementals.append(ElementalOperator(coef,[g1,g2,g3,g4],[q2,q3,q4,q5]))
        
#insertion=Operator(elementals)
#s=''
#for e in insertion.elementals:
#    s+=str(e)+'\\\\'
#print(s.replace('*',''))

creation=Operator(elementals)

In [5]:
res=contract(annihilate,creation)
print(len(res))
res[0].coef

80


-0.4444444444444444

In [12]:
import copy

def latexStr(gamma):
    tmp=gamma.name.split('_')
    if(tmp[0]=='(1-\\gamma)') or (tmp[0]=='(1+\\gamma)'):
        return tmp[0] + '_{' + tmp[1][:-1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    else:
        return str(gamma)
  
def platexStr(p):
    return p.name + '(' + p.ti.replace('t','x') + p.ti + '\mid ' + p.tf.replace('t','x') + p.tf + ')' + '_{\\substack{' + p.left_indices.s + '\\\\' + p.left_indices.c + '}' +  '\\substack{' + p.right_indices.s + '\\\\' + p.right_indices.c + '}}'
    
total=copy.deepcopy(res)

print(len(total))

for d in total:
    for p in d.props:
        p.name = 'D^{-1}'
        
for i in range(len(total)-1,-1,-1):
    for j in range(len(total)-1,-1,-1):
        if(i!=j):
            if((total[i].commuting==total[j].commuting) and (total[i].props==total[j].props)):
                total[i].coef+=total[j].coef
                total.pop(j)
                break
for i in range(len(total)-1,-1,-1):
    if(total[i].coef==0):
        total.pop(i)
        
print(len(total))

80
48


In [13]:
for i,d in enumerate(total):
    #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}='+str(d.coef) + ' '
        for p in d.props:
            tmp+=platexStr(p)
        for c in d.commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')

d_{0}=0.4444444444444444 D^{-1}(x_1t_1\mid x_3t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}D^{-1}(x_0t_0\mid x_2t_2)_{\substack{s_6\\c_3}\substack{s_2\\c_1}}D^{-1}(x_2t_2\mid x_1t_1)_{\substack{s_3\\c_2}\substack{s_4\\c_3}}D^{-1}(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}(1-\gamma_{\mu})_{s_2 s_3 }(1-\gamma_{\nu})_{s_4 s_5 }U_{\mu}(x_2)_{c_1 c_2 }U_{\nu}(x_1)_{c_3 c_4 }\gamma_5_{s_0 s_1 }\gamma_5_{s_6 s_7 }\\
d_{1}=0.4444444444444444 D^{-1}(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_2}\substack{s_0\\c_0}}D^{-1}(x_1t_1\mid x_2t_2)_{\substack{s_5\\c_4}\substack{s_2\\c_1}}D^{-1}(x_0t_0\mid x_1t_1)_{\substack{s_6\\c_3}\substack{s_4\\c_3}}D^{-1}(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}(1-\gamma_{\mu})_{s_2 s_3 }(1-\gamma_{\nu})_{s_4 s_5 }U_{\mu}(x_2)_{c_1 c_2 }U_{\nu}(x_1)_{c_3 c_4 }\gamma_5_{s_0 s_1 }\gamma_5_{s_6 s_7 }\\
d_{2}=-0.4444444444444444 D^{-1}(x_1t_1\mid x_3t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}D^{-1}(x_0t_0\mid x_2t_2)_{\substack{s_6\\c_3}\s

In [14]:
diagrams = {}

for i,d in enumerate(total):
    top = []
    d_str='d_' + str(i) + ':  '
    for p in d.props:
        d_str+=p.tf + '->' + p.ti + '   '
        top.append([p.tf.split('_')[1],p.ti.split('_')[1]])
    diagrams[i]=top
    print(d_str)

d_0:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_1:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_2:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_3:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_4:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_5:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_6:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_7:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_8:  t_3->t_2   t_2->t_0   t_1->t_3   t_0->t_1   
d_9:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_10:  t_3->t_2   t_2->t_0   t_1->t_3   t_0->t_1   
d_11:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_12:  t_3->t_2   t_2->t_0   t_1->t_3   t_0->t_1   
d_13:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_14:  t_3->t_2   t_2->t_0   t_1->t_3   t_0->t_1   
d_15:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_16:  t_3->t_1   t_2->t_3   t_1->t_0   t_0->t_2   
d_17:  t_3->t_1   t_2->t_2   t_1->t_0   t_0->t_3   
d_18:  t_3->t_1   t_2->t_3   t_1->t_0   t_0->t_2   
d_19:  t_3->t_1   t_2-

In [15]:
refTopologies={}
refTopologies['A']=[['1','0'],['0','2'],['2','3'],['3','1']]
refTopologies['A-bwd']=[['0','1'],['1','3'],['3','2'],['2','0']]
refTopologies['B']=[['1','0'],['0','3'],['3','2'],['2','1']]
refTopologies['B-bwd']=[['0','1'],['1','2'],['2','3'],['3','0']]
refTopologies['C']=[['2','0'],['0','3'],['3','1'],['1','2']]
refTopologies['C-bwd']=[['0','2'],['2','1'],['1','3'],['3','0']]
refTopologies['D']=[['0','3'],['3','0'],['2','1'],['1','2']]
refTopologies['El']=[['2','2'],['3','1'],['1','0'],['0','3']]
refTopologies['El-bwd']=[['2','2'],['1','3'],['3','0'],['0','1']]
refTopologies['Er']=[['1','1'],['3','2'],['2','0'],['0','3']]
refTopologies['Er-bwd']=[['1','1'],['2','3'],['3','0'],['0','2']]
refTopologies['F']=[['0','3'],['3','0'],['1','1'],['2','2']]

In [16]:
def swap(list, i, j):
    list[i], list[j] = list[j], list[i]
    return list

for idx,d in diagrams.items():
    for p in range(len(d)):
        if d[p][1] != d[(p+1)%len(d)][0]:
            for pp in range(p,len(d)):
                if d[p][1] == d[pp][0]:
                    swap(d,(p+1)%len(d),pp)

In [17]:
from src.wick.utilities import permutations

def cyclic_permutations(list):
    return [[list[i-j] for i in range(len(list))] for j in range(len(list))]

print('\\begin{tabular}{c | c}')
print('d & Diagrams\\\\')
print('\\hline')
for idx,d in diagrams.items():
    for label, top in refTopologies.items():
        if d in permutations(top):
            print('{} & {}\\\\'.format(idx,label[0]))
print('\\end{tabular}')

\begin{tabular}{c | c}
d & Diagrams\\
\hline
0 & C\\
1 & B\\
2 & C\\
3 & B\\
4 & C\\
5 & B\\
6 & C\\
7 & B\\
8 & A\\
9 & E\\
10 & A\\
11 & E\\
12 & A\\
13 & E\\
14 & A\\
15 & E\\
16 & A\\
17 & E\\
18 & A\\
19 & E\\
20 & A\\
21 & E\\
22 & A\\
23 & E\\
24 & E\\
25 & B\\
26 & E\\
27 & C\\
28 & D\\
29 & F\\
30 & E\\
31 & B\\
32 & E\\
33 & C\\
34 & D\\
35 & F\\
36 & E\\
37 & B\\
38 & E\\
39 & C\\
40 & D\\
41 & F\\
42 & E\\
43 & B\\
44 & E\\
45 & C\\
46 & D\\
47 & F\\
\end{tabular}
